# Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how should it be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. User knows that he doesn't know the correct spelling OR he wants to get the results that follow some known pattern, so he uses so called wildcards - queries like 'retr*val';
2. User doesn't know the correct spelling OR he doesn't care OR he's in a rush OR he expects his mistakes will be corrected OR your option, so he makes mistakes and we need to handle them using:

    2.1. Simple spellchecker by Peter Norvig;
    
    2.2. Phonetic correction by means of Soundex algorithm;
    
    2.3. Trigrams with Jaccard coefficient.

## 1. Handling wildcards

We will handle wildcard queries using k-grams. K-grams is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

In [21]:
import tqdm 

In [1]:
import urllib.request
data_url = "https://raw.githubusercontent.com/hsu-ai-course/hsu.ai/master/code/datasets/nlp/facts.txt"
file_name= "facts.txt"
# urllib.request.urlretrieve(data_url, file_name)

facts = []
with open(file_name,encoding="windows-1251") as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n')[len(str(cnt))+2:])
print(*facts[-5:], sep='\n')

Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
There are more stars in space than there are grains of sand on every beach in the world.
For every human on Earth there are 1.6 million ants.
The total weight of all those ants, however, is about the same as all the humans.
On Jupiter and Saturn it rains diamonds.


In [45]:
import nltk
from collections import Counter
import re
from itertools import islice, tee

def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)   
    inverted_index = {}
    for doc_id ,doc in enumerate(documents,0):
        words = nltk.word_tokenize(doc.lower())
        count = Counter(words)
        for pair in count.items():
            if pair[0] not in inverted_index.keys():
                inverted_index[pair[0]] = [pair[1],(doc_id,pair[1])]
            else:
                inverted_index[pair[0]][0] += pair[1]
                inverted_index[pair[0]].append((doc_id,pair[1]))
    
    return inverted_index

def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}
    padded_words = ["$"+i+"$" for i in inverted_index.keys()]
    all_words = list(inverted_index.keys())
    
    for padded_w in tqdm.tqdm_notebook(padded_words):
        #k_grams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(padded_w, k))))
        k_grams = nltk.ngrams(padded_w, k)
        for k_gram in k_grams:
            k_gram = "".join(k_gram)
            res_words_with_kgram = [i[1:-1] for i in padded_words if k_gram in i ]
            k_gram_index[k_gram] = res_words_with_kgram
    
    return k_gram_index

def generate_wildcard_options(wildcard, k_gram_index,k=3): 
    
    if (wildcard[0] == "*"): wildcard = wildcard+"$"
    elif (wildcard[-1] == "*"): wildcard = "$"+wildcard
    else : wildcard = "$"+wildcard+"$"
    
    wildcard_splits = wildcard.split("*")

    k = len(list(k_gram_index.keys())[0]) #3
    kgrams = list(k_gram_index.keys())
    answer = None
    for index, w_c in enumerate(wildcard_splits):
        if len(w_c) < 3:
            if index + 1 < len(wildcard_splits):
                while len(w_c) < k:
                    w_c += '.'
            else:
                while len(w_c) < k:
                    w_c = '.' + w_c
                    
        for i in range(len(w_c) - k + 1):
            temp_res = set()
            kgram = w_c[i:i + k]
            kgram = kgram.replace("$", r"\$")
            rexpr = re.compile(kgram)
            results = filter(rexpr.match, kgrams)
            
            for item in results:
                temp_res = temp_res.union(set(k_gram_index[item]))
            if answer is None:
                answer = temp_res
            answer = answer.intersection(temp_res)

    n = len(re.sub("[*$]","",wildcard))

    return list(filter(lambda w : n <= len(w),answer))

def search_wildcard(wildcard, k_gram_index, index, docs):
    """retrive list of documnets (facts) that contain words matching wildcard"""
    
    wildcard_options = generate_wildcard_options(wildcard,k_gram_index)
    results = []
    for option in wildcard_options:
        #get the documents containing this word
        doc_list = index.get(option, None)
        if doc_list != None:
            results += [docs[i[0]] for i in doc_list[1:]]
            
    return results


### 1.2 Tests

In [4]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

# wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
wildcard_options = generate_wildcard_options(wildcard, k_gram_index)
print(wildcard_options)
assert(len(wildcard_options) >= 3)

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['reduced', 'received', 'recorded']
A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you’ll feel thirsty. If it’s reduced by 10%, you’ll die.
More than 50% of the people in the world have never made or received a telephone call.
The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.


## 2. Handling typos

### 2.1 Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [5]:
# !curl -o "github-typo-corpus.v1.0.0.jsonl.gz" "https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz"
# !gzip -d "github-typo-corpus.v1.0.0.jsonl.gz"

# !wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
# !gunzip -k ./github-typo-corpus.v1.0.0.jsonl.gz

In [6]:
# !pip3 install jsonlines
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [8]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### Build a dataset vocabulary
We will need it for Norvig's spellchecker as well as for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [33]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    #words = nltk.word_tokenize(re.sub("[^a-zA-Z0-9]+"," ",sent))
    words = nltk.word_tokenize(sent)
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [34]:
import nltk
from collections import Counter
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
print(f'Vocabulary has {len(vocabulary)} words')

Vocabulary has 63204 words


In [32]:
import nltk
from collections import Counter
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
print(f'Vocabulary has {len(vocabulary)} words')

Vocabulary has 76802 words


In [11]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6162), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42018), ('can', 26006), ('disable', 529), ('comments', 359), ('for', 44724), ('the', 206931)]


### 2.2 Implement context-independent spellcheker ##

1. Write code to compute editorial distance

1. [Norvig's corrector](https://norvig.com/spell-correct.html)

1. [Soundex](https://en.wikipedia.org/wiki/Soundex)

1. Trigrams with Jaccard coefficient.

#### Editorial distance

Frequently used distance measure between two character sequences. We will use this distance to sort Soundex search results.

In [12]:
def edit_dist1(s1, s2) -> int:
    """REF : https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance"""
    if len(s1) > len(s2):
        s1,s2 = s2,s1
    res_distances = range(len(s1) + 1)
    
    for index_w2,char_w2 in enumerate(s2):
        temp_distances = [index_w2+1]
        
        for index_w1,char_w1 in enumerate(s1):
            if char_w1 == char_w2 : temp_distances.append(res_distances[index_w1])
            else : temp_distances.append(1 + min((temp_distances[-1],res_distances[index_w1], res_distances[index_w1+1])))
            
        res_distances = temp_distances
        
    return res_distances[-1]

In [13]:
def edit_dist(s1, s2) -> int:
    """Recursive version of Levenshtein distance"""
    if s1 == "":
        return len(s2)
    if s2 == "":
        return len(s1)
    if s1[-1] == s2[-1]:
        cost = 0
    else:
        cost = 1
    
    return min((edit_dist(s1[:-1], s2) + 1,edit_dist(s1, s2[:-1]) + 1, edit_dist(s1[:-1], s2[:-1]) + cost))

#### Tests

In [14]:
assert edit_dist("korrectud", "corrected") == 2, "Edit distance is computed incorrectly"
assert edit_dist("soem", "some") == 2, "Edit distance is computed incorrectly"
assert edit_dist("one", "one") == 0, "Edit distance is computed incorrectly"

#### Norvig's spellchecker

In [15]:
# From : https://norvig.com/spell-correct.html 

def candidates(word): 
    "possible spelling corrections for word"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    return set(w for w in words if w in vocabulary)

def edits1(word):
    "All edits that are one edit away from given word"
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def fix_typo_norvig(word, vocabulary) -> str:
    n = sum(vocabulary.values())
    
    return max(candidates(word), key=lambda w : vocabulary[w] / n)


#### Tests

In [16]:
assert fix_typo_norvig("korrectud",vocabulary) == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_norvig("speling",vocabulary) == "spelling", "Norvig's correcter doesn't work"

#### Soundex 

For cases when the exact spelling is unknown, phonetic algorithms such as Soundex can be very helpful - they allow user to type a word the way he thinks it should sound, and then suggest the corrrect version. Go through *chapter 3.4* to understand how Soundex algorithm works.

In [17]:
def produce_soundex_code(word):
    """Generate soundex code for any given word"""
    #Retain the first letter of the name added to result
    result = word[0].upper()
    
    #part of step 1 , Avoid removing vowels immediately 
    word = re.sub('[hw]', '', word, flags=re.I).lower()

    # Step 2 & 3 : mapping letters to respective number code  
    coded_word = re.sub('[bfpv]+', '1', word)
    coded_word = re.sub('[cgjkqsxz]+', '2', coded_word)
    coded_word = re.sub('[dt]+', '3', coded_word)
    coded_word = re.sub('l+', '4', coded_word)
    coded_word = re.sub('[mn]+', '5', coded_word)
    coded_word = re.sub('r+', '6', coded_word)

    # Remove the first letter
    coded_word = coded_word[1:]

    # remove all vowels and y's now
    coded_word = re.sub('[yaeiou]','', coded_word)

    # Result should be made of the first letter plus 3 other codes
    result += coded_word[0:3] if len(coded_word) > 3 else (coded_word + ('0'*(3-len(coded_word))))

    return result


def build_soundex_index(dictionary):
    """build soundex index for dictionary words.
    Input : vocabulary dictionary of original words
    Output : dictionary -> {'code1': ['word1_with_code1', 'word2_with_code1', ...]  } 
    """
    soundex_index = {}
    
    for w in dictionary.keys():
        word_code = produce_soundex_code(w)
        if word_code not in soundex_index:
            soundex_index[word_code] = [w]
        else:
            soundex_index[word_code].append(w)
    
    return soundex_index

def fix_typo_soundex(word, soundex_index) -> list:
    """ 
    retrieve words from vocabulary that match with result by soundex fingerprint
    NB : ordered results by editorial distance
    """
    word_code = produce_soundex_code(word)
    options = soundex_index.get(word_code,None)
    if options == None: return []
    else: return sorted(options,key= lambda x : edit_dist(x,word))
    

#### Tests

In [18]:
soundex_index = build_soundex_index(vocabulary)

code1 = produce_soundex_code("britney")
code2 = produce_soundex_code("breatany")
print(code1, code2)
assert code1 == code2

print(fix_typo_soundex("enouhg", soundex_index))
assert "enough" in fix_typo_soundex("enouhg", soundex_index), "Assert soundex failed"

B635 B635
['enough', 'ensue', 'eng', 'enjoy', 'emoji', 'enqueue', 'ens', 'enc', 'emojii', 'enki', 'enso', 'enzo', 'enwiki', 'emesh', 'emg', 'emacs', 'emc', 'emas', 'euank', 'enmasse', 'emac', 'emmc', 'emgo']


#### Trigrams with Jaccard coefficient

In [19]:
def fix_typo_kgram(word, k_gram_index) -> list:
    #TODO return best matches with respect to Jaccard index
    k = len(k_gram_index.keys()[0])
    k_grams = nltk.ngrams("$"+word+"$", k)
    res = []
    for k_gram in k_grams:
        k_gram = "".join(k_gram)
        res += k_gram_index.get(k_gram,[])
    if len(res) != 0 : return sorted(res,key= lambda w : nltk.jaccard_distance(set(word), set(w)))
    else: return []

In [3]:
k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])

#### Tests

In [16]:
k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'enought', 'eno', 'endogenous', 'enomem', 'enospc', 'enosys', 'enormous', 'renounce', 'en', 'exogenous', 'uh', 'enormously', 'homogenous', 'env', 'end', 'huh', 'hetrogenous', 'enh', 'ens']


### 2.3 Estimate quality

In [ ]:
norvig, soundex, kgram = 0, 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, ss, sk = src.lower(), src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, ws, wk = fix_typo_norvig(word), \
                         fix_typo_soundex(word, soundex_index)[0], \
                         fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            ss = ss.replace(word, ws)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    soundex += int(ss == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"Soundex accuracy ({soundex}) = {soundex / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")

# Norvig accuracy (2346) = 0.2346
# Soundex accuracy (1673) = 0.1673
# k-gram accuracy (1566) = 0.1566

In [4]:
!rm github-typo-corpus.v1.0.0.jsonl